## **Hybrid Agent Graph (Simulate High Severity Weather level check with dummy weather and Social Monitoring data):**


## **Note:** Below, only details are given for nodes or functions, which are not described above

##  **1. Installation of Required Packages**

In [ ]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain langchain_google_genai langchain_community schedule

%%capture --no-stderr
%pip install -U langgraph langsmith langchain langchain_google_genai langchain_community schedule

## **2. Call Credentials and Set API Keys**

In [ ]:
import os
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')
API_KEY = userdata.get('W_API_KEY')
NEWS_API_KEY = userdata.get('NEWS_API_KEY')
os.environ["API_KEY"] = API_KEY
os.environ["GOOGLE_API_KEY"] = gemini_api_key
os.environ['NEWS_API_KEY'] = NEWS_API_KEY

In [ ]:
import os
print("API_KEY:", os.getenv("API_KEY"))


API_KEY: f04579c32f6278796260a9e0b0c65fca


In [ ]:
import requests
import os

def get_weather_news_for_city(city: str, api_key: str) -> str:
    """
    Searches for recent weather-related news articles about a specific city using NewsAPI.
    Filters results to include only articles that mention both the city and severe weather terms.
    """
    # Define severe weather-related keywords
    keywords = ["storm", "flood", "heatwave", "tornado", "hail", "lightning", "thunderstorm", "weather alert", "evacuation", "emergency"]
    keyword_query = " OR ".join([f'"{kw}"' for kw in keywords])
    search_query = f'"{city}" AND ({keyword_query})'

    # Prepare the request
    url = (
        f"https://newsapi.org/v2/everything?"
        f"q={requests.utils.quote(search_query)}"
        f"&language=en&sortBy=publishedAt&pageSize=10&apiKey={api_key}"
    )

    # Make request
    response = requests.get(url)
    if response.status_code != 200:
        return f"❌ Failed to fetch news: {response.status_code} - {response.text}"

    articles = response.json().get("articles", [])
    city_lower = city.lower()

    # Filter for strong match to city name and weather content
    relevant_articles = []
    for article in articles:
        title = article.get("title", "")
        desc = article.get("description", "")
        content = article.get("content", "")
        combined_text = f"{title} {desc} {content}".lower()

        if city_lower in combined_text:
            relevant_articles.append({
                "title": title,
                "source": article.get("source", {}).get("name", ""),
                "url": article.get("url", "")
            })

    # Format results
    if relevant_articles:
        report = [f"📍 Weather News for {city.title()}:"]
        for a in relevant_articles:
            report.append(f"📰 {a['title']} — {a['source']}\n🔗 {a['url']}")
        return "\n\n".join(report)
    else:
        return f"✅ {city.title()} is currently calm. No recent severe weather news reported."

# 🔐 Set your NewsAPI key
os.environ["NEWS_API_KEY"] = "5935628fb485429982f522f2bb117d4c"  # Replace this or set externally

# ✅ Run the search
city = "Houston"
api_key = os.getenv("NEWS_API_KEY")
print(get_weather_news_for_city(city, api_key))


📍 Weather News for Houston:

📰 Installment Loans for Bad Credit Online Guaranteed Instant Approval From Best Direct Lenders 2025 — GlobeNewswire
🔗 https://www.globenewswire.com/news-release/2025/05/01/3072645/0/en/Installment-Loans-for-Bad-Credit-Online-Guaranteed-Instant-Approval-From-Best-Direct-Lenders-2025.html


## **3. Standard Library Imports**

In [ ]:
import os
import random
import requests
import schedule
import time
from typing import Dict, TypedDict, Union, List, Literal
import json
from datetime import datetime
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

## **4. Defining an LLM and a State Class**

In [ ]:
# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

class WeatherState(TypedDict):
    city: str
    weather_data: Dict
    disaster_type: str
    severity: str
    response: str
    messages: List[Union[SystemMessage, HumanMessage, AIMessage]]
    alerts: List[str]
    social_media_reports: List[str]
    human_approved: bool
    test_mode: bool  # <-- add this

## **5. Define Node Functions**

#### **(a) Fetching Weather Data**

In [ ]:
def get_weather_data(state: WeatherState) -> Dict:
    """Fetch short-term forecast weather data from OpenWeatherMap API or use simulated data in test mode"""

    if state.get("test_mode") and state['weather_data']:
        return {
            **state,
            "messages": state["messages"] + [
                SystemMessage(content=f"🧪 Using simulated forecast data for {state['city']} (test mode)")
            ]
        }

    BASE_URL = "http://api.openweathermap.org/data/2.5/forecast"
    API_KEY = os.getenv("API_KEY")

    if not API_KEY:
        return {
            **state,
            "messages": state["messages"] + [SystemMessage(content="❌ API_KEY not found in environment variables.")]
        }

    request_url = f"{BASE_URL}?q={state['city']}&appid={API_KEY}&units=metric"

    try:
        response = requests.get(request_url)
        response.raise_for_status()
        data = response.json()

        # Use next 6 hours forecast (2 intervals of 3 hours)
        next_hours = data.get('list', [])[:2]

        weather_conditions = [block['weather'][0]['description'] for block in next_hours]
        wind_speeds = [block['wind']['speed'] for block in next_hours]
        temps = [block['main']['temp'] for block in next_hours]
        humidities = [block['main']['humidity'] for block in next_hours]
        pressures = [block['main']['pressure'] for block in next_hours]
        clouds = [block['clouds']['all'] for block in next_hours]

        forecast_summary = {
            "weather": ", ".join(set(weather_conditions)),  # Combine unique descriptions
            "wind_speed": max(wind_speeds),
            "temperature": max(temps),
            "humidity": max(humidities),
            "pressure": min(pressures),
            "cloud_cover": max(clouds),
            "sea_level": "N/A"  # Usually not in forecast data
        }

        return {
            **state,
            "weather_data": forecast_summary,
            "messages": state["messages"] + [SystemMessage(content=f"✅ Forecast data fetched for {state['city']}")]
        }

    except Exception as e:
        fallback_data = {
            "weather": "N/A",
            "wind_speed": "N/A",
            "cloud_cover": "N/A",
            "sea_level": "N/A",
            "temperature": "N/A",
            "humidity": "N/A",
            "pressure": "N/A"
        }

        return {
            **state,
            "weather_data": fallback_data,
            "messages": state["messages"] + [SystemMessage(content=f"❌ Failed to fetch forecast data: {str(e)}")]
        }


#### **(b) Social Media Monitoring**


In [ ]:
import random
import requests
from langchain_core.messages import SystemMessage

import requests

def get_weather_news_for_city(city: str, api_key: str) -> list:
    """
    Strictly fetch weather-related news headlines for a city.
    Filters aggressively to remove non-weather content.
    """
    # Very weather-specific keywords (high precision)
    weather_keywords = [
        "severe weather", "flash flood", "thunderstorm", "hailstorm", "tornado", "flood warning",
        "weather alert", "storm damage", "strong winds", "heatwave", "blizzard", "snowfall",
        "extreme temperatures", "wind advisory", "winter storm", "rainfall", "weather forecast"
    ]

    # Build search query
    keyword_query = " OR ".join([f'"{kw}"' for kw in weather_keywords])
    query = f'"{city}" AND ({keyword_query})'

    url = (
        f"https://newsapi.org/v2/everything?"
        f"q={requests.utils.quote(query)}"
        f"&language=en&sortBy=publishedAt&pageSize=20&apiKey={api_key}"
    )

    response = requests.get(url)
    if response.status_code != 200:
        return [f"❌ Failed to fetch news: {response.status_code} - {response.text}"]

    articles = response.json().get("articles", [])
    city_lower = city.lower()
    relevant = []

    for article in articles:
        title = article.get("title", "").lower()
        description = article.get("description", "").lower()
        content = article.get("content", "").lower()
        full_text = f"{title} {description} {content}"

        # Skip if city is not clearly mentioned
        if city_lower not in full_text:
            continue

        # Strict filter: must match a true weather phrase
        matched = False
        for kw in weather_keywords:
            if kw in full_text:
                matched = True
                break

        if not matched:
            continue

        # Final strict inclusion
        headline = article.get("title", "")
        source = article.get("source", {}).get("name", "")
        url = article.get("url", "")
        relevant.append(f"📰 {headline} — {source}\n🔗 {url}")

    return relevant if relevant else [f"✅ {city.title()} is currently calm. No severe weather news reported."]




def social_media_monitoring(state: WeatherState) -> WeatherState:
    """Uses simulated data in test mode, or fetches real news headlines using NewsAPI when live."""
    city = state["city"]
    messages = []
    reports = []

    if state.get("test_mode"):
        simulated_reports = [
            "Local reports of rising water levels and minor flooding.",
            "High winds causing power outages in parts of the city.",
            "Citizens reporting high temperatures and increased heat discomfort.",
            "Social media reports indicate severe storm damage in local infrastructure.",
            "Reports of traffic disruptions due to heavy rain.",
            "No unusual social media reports related to the weather at this time."
        ]
        report = random.choice(simulated_reports)
        reports.append(report)
        messages.append(SystemMessage(content=f"🧪 Simulated report added: {report}"))

    else:
        api_key = os.getenv("NEWS_API_KEY")
        if not api_key:
            fallback = f"❌ NEWS_API_KEY not set. Cannot fetch real weather news for {city}."
            reports.append(fallback)
            messages.append(SystemMessage(content=fallback))
        else:
            try:
                news_reports = get_weather_news_for_city(city, api_key)
                reports.extend(news_reports)
                messages.append(SystemMessage(content=f"✅ Pulled {len(news_reports)} news headlines for {city}."))
            except Exception as e:
                error_msg = f"❌ Error fetching news for {city}: {str(e)}"
                reports.append(error_msg)
                messages.append(SystemMessage(content=error_msg))

    return {
        **state,
        "social_media_reports": state["social_media_reports"] + reports,
        "messages": state["messages"] + messages
    }


#### **(c) Analyzing Disaster Type**


In [ ]:
def analyze_disaster_type(state: WeatherState) -> WeatherState:
    """Analyze weather data to identify potential disasters"""
    weather_data = state["weather_data"]
    prompt = ChatPromptTemplate.from_template(
        "Based on the following weather conditions, identify if there's a potential weather disaster.\n"
        "Weather conditions:\n"
        "- Description: {weather}\n"
        "- Wind Speed: {wind_speed} m/s\n"
        "- Temperature: {temperature}°C\n"
        "- Humidity: {humidity}%\n"
        "- Pressure: {pressure} hPa\n"
        "Categorize into one of these types: Hurricane, Flood, Heatwave, Severe Storm, Winter Storm, or No Immediate Threat"
    )

    try:
        chain = prompt | llm
        disaster_type = chain.invoke(weather_data).content
        return {
            **state,
            "disaster_type": disaster_type,
            "messages": state["messages"] + [SystemMessage(content=f"Disaster type identified: {disaster_type}")]
        }
    except Exception as e:
        return {
            **state,
            "disaster_type": "Analysis Failed",
            "messages": state["messages"] + [SystemMessage(content=f"Failed to analyze disaster type: {str(e)}")]
        }


#### **(d) Assess Severity Level**


In [ ]:
def assess_severity(state: WeatherState) -> WeatherState:
    """Assess the severity of the identified weather situation"""
    weather_data = state["weather_data"]
    prompt = ChatPromptTemplate.from_template(
        "Given the weather conditions and identified disaster type '{disaster_type}', "
        "assess the severity level. Consider:\n"
        "- Weather: {weather}\n"
        "- Wind Speed: {wind_speed} m/s\n"
        "- Temperature: {temperature}°C\n"
        "Respond with either 'Critical', 'High', 'Medium', or 'Low'."
    )

    try:
        chain = prompt | llm
        severity = chain.invoke({
            "disaster_type": state["disaster_type"],
            "weather": weather_data.get("weather", "unknown"),
            "wind_speed": weather_data.get("wind_speed", 0),
            "temperature": weather_data.get("temperature", 0)
        }).content.strip()

        return {
            **state,
            "severity": severity,
            "messages": state["messages"] + [SystemMessage(content=f"Severity assessed as: {severity}")]
        }
    except Exception as e:
        return {
            **state,
            "severity": "Assessment Failed",
            "messages": state["messages"] + [SystemMessage(content=f"Failed to assess severity: {str(e)}")]
        }



#### **(e) Emergency Response**

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

local_time = datetime.now(ZoneInfo("America/Chicago"))

def emergency_response(state):
    """Generate emergency response plan with dynamic date and correct city injection"""
    today = local_time.strftime('%Y-%m-%d %H:%M:%S')
    disaster_type = state.get("disaster_type", "Unknown Disaster")
    severity = state.get("severity", "Unknown")
    city = state.get("city", "Unknown Location")

    try:
        prompt = ChatPromptTemplate.from_template(
            "Create an emergency response plan for a {disaster_type} situation "
            "with {severity} severity level in {city} on {date}. "
            "Ensure the city name {city} appears throughout the response, especially in headings like 'Date', 'Incident', and 'Action Plan'."
        )
        chain = prompt | llm
        response = chain.invoke({
            "disaster_type": disaster_type,
            "severity": severity,
            "city": city,
            "date": today
        }).content

        return {
            **state,
            "response": response,
            "messages": state["messages"] + [SystemMessage(content="Emergency response plan generated")]
        }

    except Exception as e:
        return {
            **state,
            "response": "Failed to generate response plan",
            "messages": state["messages"] + [SystemMessage(content=f"Failed to generate emergency response: {str(e)}")]
        }


#### **(f) Civil Defense Response**

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

local_time = datetime.now(ZoneInfo("America/Chicago"))



def civil_defense_response(state):
    """Generate civil defense response plan with dynamic date and correct city injection"""
    today = local_time.strftime('%Y-%m-%d %H:%M:%S')
    disaster_type = state.get("disaster_type", "Unknown Disaster")
    severity = state.get("severity", "Unknown")
    city = state.get("city", "Unknown Location")

    try:
        prompt = ChatPromptTemplate.from_template(
            "Create a civil defense response plan for a {disaster_type} situation "
            "with {severity} severity level in {city} on {date}. "
            "Mention the city name {city} in all major sections including 'Date', 'Incident', and 'Preparedness Summary'."
        )
        chain = prompt | llm
        response = chain.invoke({
            "disaster_type": disaster_type,
            "severity": severity,
            "city": city,
            "date": today
        }).content

        return {
            **state,
            "response": response,
            "messages": state["messages"] + [SystemMessage(content="Civil defense response plan generated")]
        }

    except Exception as e:
        return {
            **state,
            "response": "Failed to generate response plan",
            "messages": state["messages"] + [SystemMessage(content=f"Failed to generate civil defense response: {str(e)}")]
        }


#### **(g) Public Works Response**

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

local_time = datetime.now(ZoneInfo("America/Chicago"))


def public_works_response(state):
    """Generate public works response plan with dynamic date and correct city injection"""
    today = local_time.strftime('%Y-%m-%d %H:%M:%S')
    disaster_type = state.get("disaster_type", "Unknown Disaster")
    severity = state.get("severity", "Unknown")
    city = state.get("city", "Unknown Location")

    try:
        prompt = ChatPromptTemplate.from_template(
            "Create a public works response plan for a {disaster_type} situation "
            "with {severity} severity level in {city} on {date}. "
            "Ensure the city name {city} is included in headings like 'Date', 'Incident Summary', and in all key bullet points."
        )
        chain = prompt | llm
        response = chain.invoke({
            "disaster_type": disaster_type,
            "severity": severity,
            "city": city,
            "date": today
        }).content

        return {
            **state,
            "response": response,
            "messages": state["messages"] + [SystemMessage(content="Public works response plan generated")]
        }

    except Exception as e:
        return {
            **state,
            "response": "Failed to generate response plan",
            "messages": state["messages"] + [SystemMessage(content=f"Failed to generate public works response: {str(e)}")]
        }


#### **(h) Data Logging**

In [ ]:


def data_logging(state: WeatherState) -> WeatherState:
    """Log weather data, disaster analysis, and response to a file."""
    log_data = {
        "timestamp": local_time.strftime('%H:%M:%S'),
        "city": state["city"],
        "weather_data": state["weather_data"],
        "disaster_type": state["disaster_type"],
        "severity": state["severity"],
        "response": state["response"],
        "social_media_reports": state["social_media_reports"]
    }

    try:
        with open("disaster_log.txt", "a") as log_file:
            log_file.write(json.dumps(log_data) + "\n")

        return {
            **state,
            "messages": state["messages"] + [SystemMessage(content="Data logged successfully")]
        }
    except Exception as e:
        return {
            **state,
            "messages": state["messages"] + [SystemMessage(content=f"Failed to log data: {str(e)}")]
        }



#### **(j) Get Human Verification**

In [ ]:
def get_human_verification(state: WeatherState) -> WeatherState:
    """Get human verification for low/medium severity alerts"""
    severity = state["severity"].strip().lower()

    # ✅ Skip human verification in test mode
    if state.get("test_mode"):
        return {
            **state,
            "human_approved": True,
            "messages": state["messages"] + [
                SystemMessage(content="✅ Skipping human approval in test mode")
            ]
        }

    if severity in ["low", "medium"]:
        print("\n" + "="*50)
        print(f"Low/Medium severity alert for {state['city']} requires human approval:")
        print(f"Disaster Type: {state['disaster_type']}")
        print(f"Current Weather: {state['weather_data']['weather']}")
        print(f"Temperature: {state['weather_data']['temperature']}°C")
        print(f"Wind Speed: {state['weather_data']['wind_speed']} m/s")
        print(f"Severity: {state['severity']}")
        print(f"Response Plan: {state['response']}")
        print("\nType 'y' to approve sending alert or 'n' to reject (waiting for input):")
        print("="*50)

        while True:
            try:
                user_input = input().lower().strip()
                if user_input in ['y', 'n']:
                    approved = user_input == 'y'
                    print(f"Human verification result: {'Approved' if approved else 'Rejected'}")
                    break
                else:
                    print("Please enter 'y' for yes or 'n' for no:")
            except Exception as e:
                print(f"Error reading input: {str(e)}")
                print("Please try again with 'y' or 'n':")

        return {
            **state,
            "human_approved": approved,
            "messages": state["messages"] + [
                SystemMessage(content=f"Human verification: {'Approved' if approved else 'Rejected'}")
            ]
        }
    else:
        return {
            **state,
            "human_approved": True,
            "messages": state["messages"] + [
                SystemMessage(content=f"Auto-approved {severity} severity alert")
            ]
        }




#### **(k) Send Email Alert**

In [ ]:
def send_email_alert(state: WeatherState) -> WeatherState:
    """Send weather alert email using secure SSL connection"""
    import smtplib
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart
    from datetime import datetime
    from langchain_core.messages import SystemMessage

    sender_email = os.getenv("SENDER_EMAIL")
    receiver_email = os.getenv("RECEIVER_EMAIL")
    password = os.getenv("EMAIL_PASSWORD")

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"Weather Alert: {state['severity']} severity weather event in {state['city']}"

    body = format_weather_email(state)
    msg.attach(MIMEText(body, 'plain'))

    try:
        print(f"Attempting to send email from {sender_email} to {receiver_email} via SSL...")
        server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
        server.set_debuglevel(1)
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        server.quit()

        severity = state["severity"].strip().lower()
        if severity in ["low", "medium"]:
            print(f"\nVerification was approved by human, Email sent to {receiver_email} successfully")
        else:
            print(f"\nEmail sent successfully for high severity alert to {receiver_email}")

        return {
            **state,
            "messages": state["messages"] + [SystemMessage(content=f"Successfully sent weather alert email for {state['city']}")],
            "alerts": state["alerts"] + [f"Email alert sent: { local_time.now()}"]
        }

    except Exception as e:
        print(f"EMAIL ERROR: {str(e)}")
        return {
            **state,
            "messages": state["messages"] + [SystemMessage(content=f"Failed to send email alert: {str(e)}")]
        }


#### **(l) Handle No Approval From Human**

In [ ]:
def handle_no_approval(state: WeatherState) -> WeatherState:
    """Handle cases where human verification was rejected"""
    print("\nVerification was not approved by human, Email not sent")

    message = (
        f"Alert not sent for {state['city']} - "
        f"Weather severity level '{state['severity']}' was deemed non-critical "
        f"by human operator and verification was rejected."
    )
    return {
        **state,
        "messages": state["messages"] + [SystemMessage(content=message)]
    }

#### **(m) Route Response for Disaster Type and Severity**

In [ ]:
def route_response(state: WeatherState) -> Literal["emergency_response", "civil_defense_response", "public_works_response"]:
    """Route to appropriate department based on disaster type and severity"""
    disaster = state["disaster_type"].strip().lower()
    severity = state["severity"].strip().lower()

    if severity in ["critical", "high"]:
        return "emergency_response"
    elif "flood" in disaster or "storm" in disaster:
        return "public_works_response"
    else:
        return "civil_defense_response"


#### **(n) Verify Approval Route after Human Verification**

In [ ]:
def verify_approval_router(state: WeatherState) -> Literal["send_email_alert", "handle_no_approval"]:
    """Route based on human approval decision"""
    return "send_email_alert" if state['human_approved'] else "handle_no_approval"



#### **(p) Format Weather Email**

In [ ]:
from datetime import datetime

def format_weather_email(state: WeatherState) -> str:
    """Format weather data and severity assessment into an email message"""
    weather_data = state["weather_data"]
    social_media_reports = "\n".join(state["social_media_reports"])

    email_content = f"""
Weather Alert for {state['city']}

Disaster Type: {state['disaster_type']}
Severity Level: {state['severity']}

Current Weather Conditions:
- Weather Description: {weather_data['weather']}
- Temperature: {weather_data['temperature']}°C
- Wind Speed: {weather_data['wind_speed']} m/s
- Humidity: {weather_data['humidity']}%
- Pressure: {weather_data['pressure']} hPa
- Cloud Cover: {weather_data['cloud_cover']}%

Social Media Reports:
{social_media_reports}

Response Plan:
{state['response']}

This is an automated weather alert generated at {local_time.strftime('%Y-%m-%d %H:%M:%S')}
"""

    if state['severity'].lower() in ['low', 'medium']:
        email_content += "\nNote: This low/medium severity alert has been verified by a human operator."

    return email_content


## **5. Creating and Compiling the Workflow**

In [ ]:
# Create the workflow
workflow = StateGraph(WeatherState)

# Add nodes
workflow.add_node("get_weather", get_weather_data)
workflow.add_node("social_media_monitoring", social_media_monitoring)
workflow.add_node("analyze_disaster", analyze_disaster_type)
workflow.add_node("assess_severity", assess_severity)
workflow.add_node("data_logging", data_logging)
workflow.add_node("emergency_response", emergency_response)
workflow.add_node("civil_defense_response", civil_defense_response)
workflow.add_node("public_works_response", public_works_response)
workflow.add_node("get_human_verification", get_human_verification)
workflow.add_node("send_email_alert", send_email_alert)
workflow.add_node("handle_no_approval", handle_no_approval)

# Add edges
workflow.add_edge("get_weather", "social_media_monitoring")
workflow.add_edge("social_media_monitoring", "analyze_disaster")
workflow.add_edge("analyze_disaster", "assess_severity")
workflow.add_edge("assess_severity", "data_logging")
workflow.add_conditional_edges("data_logging", route_response)

workflow.add_edge("civil_defense_response", "get_human_verification")
workflow.add_edge("public_works_response", "get_human_verification")
workflow.add_conditional_edges("get_human_verification", verify_approval_router)
workflow.add_edge("emergency_response", "send_email_alert")
workflow.add_edge("send_email_alert", END)
workflow.add_edge("handle_no_approval", END)

workflow.set_entry_point("get_weather")

# Compile the workflow
app = workflow.compile()

##  **6 Running the Weather Emergency System**

#### **(a) run_weather_emergency_system(city: str):**

In [ ]:
def run_weather_emergency_system(city: str):
    """Initialize and run the weather emergency system for a given city"""
    initial_state = {
        "city": city,
        "weather_data": {},
        "disaster_type": "",
        "severity": "",
        "response": "",
        "messages": [],
        "alerts": [],
        "social_media_reports": [],
        "human_approved": False,
        "test_mode": False  # <-- Important
    }

    try:
        result = app.invoke(initial_state)
        print(f"Completed weather check for {city}")
        return result
    except Exception as e:
        print(f"Error running weather emergency system: {str(e)}")

#### **(b) Get Simulated Weather Data:**

In [ ]:
def get_simulated_weather_data(scenario: str = "high") -> Dict:
    """Generate simulated weather data for testing different scenarios"""
    scenarios = {
        "high": {
            "weather": "severe thunderstorm with heavy rainfall and strong winds",
            "wind_speed": 32.5,  # Increased for high severity
            "cloud_cover": 95,
            "sea_level": 1015,
            "temperature": 35.5,  # Higher temperature
            "humidity": 90,
            "pressure": 960  # Low pressure indicating severe weather
        },
        "medium": {
            "weather": "moderate rain with gusty winds",
            "wind_speed": 15.2,
            "cloud_cover": 75,
            "sea_level": 1012,
            "temperature": 22.3,
            "humidity": 70,
            "pressure": 1005
        },
        "low": {
            "weather": "light drizzle",
            "wind_speed": 8.5,
            "cloud_cover": 45,
            "sea_level": 1013,
            "temperature": 20.1,
            "humidity": 60,
            "pressure": 1015
        }
    }

    return scenarios.get(scenario.lower(), scenarios["medium"])


#### **(c) Run Weather Emergency System Test**

In [ ]:
def run_weather_emergency_system_test(city: str, scenario: str = "high") -> dict:
    """Test the weather emergency system with simulated data"""
    initial_state = {
        "city": city,
        "weather_data": get_simulated_weather_data(scenario),
        "disaster_type": "",
        "severity": "",
        "response": "",
        "messages": [],
        "alerts": [],
        "social_media_reports": [],
        "human_approved": False,
        "test_mode": True  # ✅ Ensure test mode is enabled
    }

    return app.invoke(initial_state)

#### **(d) Run Main Function**


### **Note:** you can change or include cities and receiver emails with yours to check that Whether email notifications are working or not

In [ ]:
import schedule
schedule.clear()

def main():
    """Main function to run the weather emergency system"""
    os.environ["SENDER_EMAIL"] = "AI.Project.625@gmail.com"
    os.environ["RECEIVER_EMAIL"] = "dyavadi324@gmail.com"
    os.environ["EMAIL_PASSWORD"] = "nlrs xhbb hjjd vidz"

    print("\nWeather Emergency Response System")
    print("1. Run normal monitoring")
    print("2. Run test with simulated data")
    choice = input("Select mode (1 or 2): ")

    if choice == "2":
        print("\nSelect test scenario:")
        print("1. High severity (no human verification needed)")
        print("2. Medium severity (requires human verification)")
        print("3. Low severity (requires human verification)")

        scenario_choice = input("Select scenario (1, 2, or 3): ")
        scenario_map = {"1": "high", "2": "medium", "3": "low"}
        scenario = scenario_map.get(scenario_choice, "medium")

        city = input("\nEnter city name for test: ").strip()
        print(f"\nRunning test scenario: {scenario.upper()} severity for {city}")
        run_weather_emergency_system_test(city, scenario)

    else:
        # Accept comma-separated cities from the user
        raw_input = input("\nEnter comma-separated cities to monitor (e.g., Austin, Illinois): ").strip()
        cities = [c.strip() for c in raw_input.split(",") if c.strip()]

        print("Monitoring these cities:", cities)

        def scheduled_check():
            """Function to perform scheduled checks for multiple cities"""
            print(f"\n🚨 Starting scheduled check at {local_time.strftime('%Y-%m-%d %H:%M:%S')}")
            for city in cities:
                try:
                    print(f"\n🚨 Checking weather conditions for: {city}")
                    run_weather_emergency_system(city)
                    time.sleep(2)
                except Exception as e:
                    print(f"Error checking {city}: {str(e)}")

        schedule.every(1).minute.do(scheduled_check)
        print("Weather Emergency Response System started.")
        print("Monitoring scheduled for every minute...")

        while True:
            try:
                schedule.run_pending()
                time.sleep(1)
            except KeyboardInterrupt:
                print("\nShutting down Weather Emergency Response System...")
                break
            except Exception as e:
                print(f"Error in main loop: {str(e)}")
                time.sleep(1)

if __name__ == "__main__":
    main()


Weather Emergency Response System
1. Run normal monitoring
2. Run test with simulated data
Select mode (1 or 2): 1

Enter comma-separated cities to monitor (e.g., Austin, Illinois): College Station
Monitoring these cities: ['College Station']
Weather Emergency Response System started.
Monitoring scheduled for every minute...

🚨 Starting scheduled check at 2025-05-02 18:46:16

🚨 Checking weather conditions for: College Station

Low/Medium severity alert for College Station requires human approval:
Disaster Type: The most likely potential weather disaster is **Flood**.  Heavy rain with high humidity increases the likelihood of significant rainfall accumulation and potential flooding. While the wind speed is moderate, it's not indicative of a hurricane or severe storm. The temperature and pressure are not extreme enough for a heatwave or winter storm.
Current Weather: heavy intensity rain
Temperature: 18.33°C
Wind Speed: 8.13 m/s
Severity: Medium
Response Plan: **College Station Public W

send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.gmail.com at your service, [34.139.18.222]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [34.139.18.222]\nSIZE 35882577\n8BITMIME\nAUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'AUTH PLAIN AEFJLlByb2plY3QuNjI1QGdtYWlsLmNvbQBubHJzIHhoYmIgaGpqZCB2aWR6\r\n'
reply: b'235 2.7.0 Accepted\r\n'
reply: retcode (235); Msg: b'2.7.0 Accepted'
send: 'mail FROM:<AI.Project.625@gmail.com> size=6542\r\n'
reply: b'250 2.1.0 OK 71dfb90a1353d-52ae403caecsm710874e0c.28 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.1.0 OK 71dfb90a1353d-52ae403caecsm710874e0c.28 - gsmtp'
send: 'rcpt TO:<dyavadi324@gmail.com>\r\n


Verification was approved by human, Email sent to dyavadi324@gmail.com successfully
Completed weather check for College Station

Shutting down Weather Emergency Response System...
